In [17]:
from dotenv import load_dotenv
import os
load_dotenv()

dbconn = os.getenv("DBCONN")

In [18]:
import psycopg

load_dotenv()
dbconn = os.getenv("DBCONN")

# establish database connection using connection string from .env
conn = psycopg.connect(dbconn)

# create a cursor
cur = conn.cursor()



In [19]:
cur.execute('''
    CREATE TABLE IF NOT EXISTS btc_data(
        id INTEGER PRIMARY KEY,
        title TEXT,
        author TEXT,
        published_at TIMESTAMP
    );
''')

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=db.rmhwmflqmrpurnexvvgh.supabase.co database=postgres) at 0x112ee5250>

In [20]:
conn.commit()

In [24]:
import psycopg
import pandas as pd
import datetime as dt

df = pd.read_csv("btc_data.csv", parse_dates=["Dates.N"])

conn = psycopg.connect(dbconn)
cur = conn.cursor()

for _, row in df.iterrows():
    # Skip rows with missing data
    if pd.isna(row["Dates.N"]):
        continue

    # Convert to Python datetime (if not already)
    published_at = pd.to_datetime(row["Dates.N"]).to_pydatetime()
    title = row["Title"]
    author = row["Author"]
    id_ = int(row["ID"])

    cur.execute(
        '''
        INSERT INTO btc_data (id, title, author, published_at)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (id) DO UPDATE SET
            title = EXCLUDED.title,
            author = EXCLUDED.author,
            published_at = EXCLUDED.published_at;
        ''',
        [id_, title, author, published_at]
    )

conn.commit()
cur.close()
conn.close()